In [ ]:
# Some code are adapted from gpt

In [1]:
!pip3 install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.1/752.1 kB 9.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output
from PIL import Image
import torch
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
trained_cls_model = YOLO('/content/drive/MyDrive/744project/step_1/train2/weights/best.pt')
trained_detect_model = YOLO('/content/drive/MyDrive/744project/step_2/train4/weights/best.pt')
trained_sign_model = YOLO('/content/drive/MyDrive/744project/step_3/train6/weights/best.pt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
def draw(image_path, bboxes_tensor, output_path):
    image = cv2.imread(image_path)
    if bboxes_tensor is None:
        cv2.imwrite(output_path, image)
        return
    if bboxes_tensor.is_cuda:
        bboxes = bboxes_tensor.cpu().numpy()
    else:
        bboxes = bboxes_tensor.numpy()
    overlay = image.copy()
    box_color = (255, 0, 0)
    hull_color = (0, 255, 0, 50)
    thickness = 2
    alpha = 0.4

    points = []
    for bbox in bboxes:
        xmin, ymin, xmax, ymax = bbox.astype(np.int32)
        points.append([xmin, ymin])
        points.append([xmin, ymax])
        points.append([xmax, ymin])
        points.append([xmax, ymax])
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), box_color, thickness)
    if not points:
        return
    points = np.array(points, dtype=np.int32)
    if points.size == 0:
        return
    hull = cv2.convexHull(points)
    # cv2.fillPoly(overlay, [hull], (0, 255, 0))
    image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)

    cv2.imwrite(output_path, image)

In [4]:
from scipy.cluster.hierarchy import linkage, fcluster

def cluster_bboxes_hierarchical(bboxes_tensor, n_clusters=2):
    if bboxes_tensor.is_cuda:
        bboxes = bboxes_tensor.cpu().numpy()
    else:
        bboxes = bboxes_tensor.numpy()
    centers_x = (bboxes[:, 0] + bboxes[:, 2]) / 2
    centers_y = (bboxes[:, 1] + bboxes[:, 3]) / 2
    widths = bboxes[:, 2] - bboxes[:, 0]
    heights = bboxes[:, 3] - bboxes[:, 1]
    features = np.column_stack((centers_x, centers_y, widths, heights))

    Z = linkage(features, method='ward')
    labels = fcluster(Z, t=n_clusters, criterion='maxclust')

    clustered_bboxes = [[] for _ in range(n_clusters)]
    for bbox, label in zip(bboxes, labels):
        clustered_bboxes[label - 1].append(bbox.tolist())

    clustered_bboxes_tensors = [torch.tensor(cluster) for cluster in clustered_bboxes]

    return clustered_bboxes_tensors


In [5]:
def filter_bboxes_by_relative_width(bboxes_tensor, ratio=0.5):
    widths = bboxes_tensor[:, 2] - bboxes_tensor[:, 0]
    threshold = widths.max() * ratio
    mask = widths >= threshold
    filtered_bboxes = bboxes_tensor[mask]
    return filtered_bboxes

In [6]:
def do_all_bboxes_overlap(bboxes_tensor):
    def calculate_iou(bbox1, bbox2):
        x_left = max(bbox1[0], bbox2[0])
        y_top = max(bbox1[1], bbox2[1])
        x_right = min(bbox1[2], bbox2[2])
        y_bottom = min(bbox1[3], bbox2[3])
        if x_right < x_left or y_bottom < y_top:
            return 0.0

        intersection_area = (x_right - x_left) * (y_bottom - y_top)

        bbox1_area = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
        bbox2_area = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])

        union_area = bbox1_area + bbox2_area - intersection_area

        iou = intersection_area / union_area
        return iou

    num_bboxes = bboxes_tensor.size(0)
    overlap_flags = [False] * num_bboxes

    for i in range(num_bboxes):
        for j in range(num_bboxes):
            if i != j:
                if calculate_iou(bboxes_tensor[i], bboxes_tensor[j]) > 0:
                    overlap_flags[i] = True
                    break
    return all(overlap_flags)

In [7]:
paths = {
    1: '/content/drive/MyDrive/744project/image/leftclosedemo.jpg',
    2: '/content/drive/MyDrive/744project/image/rightclosedemo.png',
    3: '/content/drive/MyDrive/744project/image/slowdemo.jpg',
    4: '/content/drive/MyDrive/744project/image/stopdemo.jpg',
    5: '/content/drive/MyDrive/744project/image/random.jpg',
    6: '/content/drive/MyDrive/744project/image/clusterdemo.jpg',
    7: '/content/drive/MyDrive/744project/image/x.png'
}
output_path = '/content/drive/MyDrive/744project/image/output.jpg'
user_input = input("Enter a number (1 for left close, 2 for right close, 3 for slow, 4 for stop, 5 for random picture, 6 for clustered picture): ")
try:
    choice = int(user_input)
    selected_path = paths.get(choice)
    if selected_path:
        print(f"You selected: {selected_path}")
    else:
        print("Invalid choice. Please enter a number between 1 and 6.")
except ValueError:
    print("Please enter a valid number.")


Enter a number (1 for left close, 2 for right close, 3 for slow, 4 for stop, 5 for random picture, 6 for clustered picture): 7
You selected: /content/drive/MyDrive/744project/image/x.png


In [10]:
res_cls = trained_cls_model.predict(source=selected_path, conf = 0.05, visualize = False, save = True)
cls = res_cls[0]
if cls.probs.data[1] > 0.3:
  res_detect = trained_detect_model.predict(source=selected_path, save = True)
  detect = res_detect[0]
  if (detect.boxes.cls == 1.0).any():
    class_1_mask = detect.boxes.cls == 1
    class_1_bboxes = detect.boxes.xyxy[class_1_mask]
    count_box = torch.sum(class_1_mask)
    if count_box >= 2:
        print("The tensor contains two or more '1's.")
        if (do_all_bboxes_overlap(class_1_bboxes) == False):
          before_cluster = filter_bboxes_by_relative_width(class_1_bboxes, 0.2)
          cluster = cluster_bboxes_hierarchical(before_cluster, n_clusters=2)
          draw(selected_path, cluster[0], output_path)
          draw(output_path, cluster[1], output_path)
            # draw(output_path, cluster[2], output_path)
        else:
          draw(selected_path, class_1_bboxes, output_path)
    else:
      draw(selected_path, class_1_bboxes, output_path)
    res_sign = trained_sign_model.predict(source=selected_path, save = True)
    sign = res_sign[0]
    if (sign.boxes.cls == 0.0).any():
      print("left lane closd")
    if (sign.boxes.cls == 1.0).any():
      print("There are unknown signs")
    if (sign.boxes.cls == 2.0).any():
      print("Right lane closed")
    if (sign.boxes.cls == 3.0).any():
      print("slow sign")
    if (sign.boxes.cls == 4.0).any():
      print("stop sign")


image 1/1 /content/drive/MyDrive/744project/image/x.png: 640x640 Work-zone-present 1.00, No-work-zone-present 0.00, 20.7ms
Speed: 13.6ms preprocess, 20.7ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/classify/predict

image 1/1 /content/drive/MyDrive/744project/image/x.png: 416x640 1 consecutive-work-zone-boundary, 26 discrete-work-zone-boundarys, 26.9ms
Speed: 1.9ms preprocess, 26.9ms inference, 1.6ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict
The tensor contains two or more '1's.

image 1/1 /content/drive/MyDrive/744project/image/x.png: 416x640 2 otherss, 27.0ms
Speed: 1.9ms preprocess, 27.0ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict2
There are unknown signs


In [ ]:
def draw_with_image(image, bboxes_tensor):
    if bboxes_tensor is None or bboxes_tensor.numel() == 0:
        return image
    if bboxes_tensor.is_cuda:
        bboxes = bboxes_tensor.cpu().numpy()
    else:
        bboxes = bboxes_tensor.numpy()

    overlay = image.copy()
    box_color = (255, 0, 0)
    thickness = 2
    alpha = 0.4
    points = []
    for bbox in bboxes:
        xmin, ymin, xmax, ymax = bbox.astype(np.int32)
        points.append([xmin, ymin])
        points.append([xmin, ymax])
        points.append([xmax, ymin])
        points.append([xmax, ymax])
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), box_color, thickness)
    if not points:
        return image
    points = np.array(points, dtype=np.int32)

    if points.size == 0:
        return image
    hull = cv2.convexHull(points)
    cv2.fillPoly(overlay, [hull], (0, 255, 0))
    image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)
    return image


In [ ]:
def process_image(image):
  res_cls = trained_cls_model.predict(source=image, conf = 0.05, visualize = False, save = False,)
  cls = res_cls[0]
  output = draw_with_image(image, None)
  if cls.probs.data[1] > 0.2:
    res_detect = trained_detect_model.predict(source=image, save = False)
    detect = res_detect[0]
    if (detect.boxes.cls == 1.0).any():
      class_1_mask = detect.boxes.cls == 1
      class_1_bboxes = detect.boxes.xyxy[class_1_mask]
      count_box = torch.sum(class_1_mask)
      if count_box >= 2:
          print("The tensor contains two or more '1's.")
          if (do_all_bboxes_overlap(class_1_bboxes) == False):
            before_cluster = filter_bboxes_by_relative_width(class_1_bboxes, 0.2)
            cluster = cluster_bboxes_hierarchical(before_cluster, n_clusters=2)
            output = draw_with_image(image, cluster[0])
            output = draw_with_image(output, cluster[1])
          else:
            draw_with_image(image, class_1_bboxes)
      else:
        draw_with_image(image,class_1_bboxes)
      res_sign = trained_sign_model.predict(source=image, save = False)
  return output

In [ ]:
output_dir = '/content/drive/MyDrive/744project/video_demo'
os.makedirs(output_dir, exist_ok=True)

video_path = '/content/drive/MyDrive/744project/video_demo/good3.MOV'
cap = cv2.VideoCapture(video_path)

original_frame_rate = cap.get(cv2.CAP_PROP_FPS)
desired_frame_rate = 10
skip_frames = int(original_frame_rate / desired_frame_rate)

output_path = os.path.join(output_dir, 'output_reduced_frame_rate.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_path, fourcc, desired_frame_rate,
                               (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

frame_counter = 0

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        if frame_counter % skip_frames == 0:
            processed_frame = process_image(frame)

            output_video.write(processed_frame)

        frame_counter += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
output_video.release()
cv2.destroyAllWindows()



0: 640x640 No-work-zone-present 1.00, Work-zone-present 0.00, 7.4ms
Speed: 37.3ms preprocess, 7.4ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 No-work-zone-present 1.00, Work-zone-present 0.00, 5.6ms
Speed: 28.9ms preprocess, 5.6ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 No-work-zone-present 1.00, Work-zone-present 0.00, 5.6ms
Speed: 28.6ms preprocess, 5.6ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 No-work-zone-present 1.00, Work-zone-present 0.00, 5.3ms
Speed: 27.3ms preprocess, 5.3ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 No-work-zone-present 1.00, Work-zone-present 0.00, 5.3ms
Speed: 27.3ms preprocess, 5.3ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 No-work-zone-present 1.00, Work-zone-present 0.00, 5.3ms
Speed: 27.7ms preprocess, 5.3ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 